# Creating Widgets from Functions - Part 2

In this notebook, we show a few extra widget options that can be created with [magicgui](https://napari.org/magicgui/).

In [1]:
from skimage.io import imread
import matplotlib.pyplot as plt
from magicgui import magicgui
import napari
from napari.utils import nbscreenshot

## Creating widgets from external functions

We can also import annotated functions from a python file and make widgets from them.

In [2]:
from my_module import preprocess_image, segment_image, subtract_bg_and_denoise, extract_features

Take a look at the local package [my_module]('my_module.py') to check the function annotations

In [3]:
image = imread("../../data/blobs.tif")
viewer = napari.Viewer()
viewer.add_image(image, name = 'blobs')

<Image layer 'blobs' at 0x277bfd62730>

In [4]:
denoise_widget = magicgui(preprocess_image)
viewer.window.add_dock_widget(denoise_widget)

In [5]:
segment_widget = magicgui(segment_image)
viewer.window.add_dock_widget(segment_widget)

In [6]:
subtract_bg_and_denoise_widget = magicgui(subtract_bg_and_denoise)
viewer.window.add_dock_widget(subtract_bg_and_denoise_widget)

In [7]:
extract_features_widget = magicgui(extract_features)
viewer.window.add_dock_widget(extract_features_widget)

## Exercise

Create your own image processing function that receives an image and returns another image. Add annotations to its arguments. Call it from code and send the output to napari.

Then, generate a widget with magicgui and add the widget to napari. Run the widget from napari. Compare your results.